In [58]:
from ts_config import GenerationConfig
import xarray
from distributed import Client
from dask.distributed import wait, TimeoutError
from uuid import uuid4
from shutil import rmtree
from pathlib import Path
import numpy as np

client = Client('localhost:8786')
client

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10, memory=465.66 GiB>

In [13]:
config = GenerationConfig(["/local1/testing_timeseries/history_files/v21.LR.BSMYLEsmbb.1970-05.001/archive.001/"], "/local1/testing_timeseries/output/")
interm_sizes = config.fit_interm_timeseries_to_memory()
batches = config.get_timeseries_batches(interm_sizes)
batches[0][0]

Sampling dataset metadata from each case group to estimate total size in memory (this may take some time)...


PosixPath('/local1/testing_timeseries/output/archive.001/atm/tseries')

In [99]:
%%time
for output_dir, group, batch_paths in batches:
    print(f"Post-processing {group} ... ", end="")
    output_dir.mkdir(parents=True, exist_ok=True)
    history_zarr_path = f"{output_dir}/tmp_hs_store.zarr"

    history_concat = xarray.open_mfdataset(batch_paths, parallel=True).chunk(dict(time=5))
    
    target_chunk_size = 250*(1024**2)
    for variable in history_concat:
        time_chunk_size = 1
        if "time" in history_concat[variable].dims:
            time_size = 1
            for index, dim in enumerate(history_concat[variable].dims):
                if dim == "time":
                    time_size = history_concat[variable].shape[index]
            smallest_time_chunk = history_concat[variable].nbytes / time_size
            if smallest_time_chunk <= 2*target_chunk_size:
                time_chunk_size = int(target_chunk_size / smallest_time_chunk)
        history_concat[variable] = history_concat[variable].chunk(dict(time=time_chunk_size))
    history_concat.to_zarr(history_zarr_path, mode="w", consolidated=True)

    dt = history_concat.time.values[1] - history_concat.time.values[0]
    if dt.days == 0:
        time_str_format = "%Y-%m-%d-%H"
    elif 30 > dt.days > 0:
        time_str_format = "%Y-%m-%d"
    else:
        time_str_format = "%Y-%m"
    
    time_start = history_concat.time.values[0].strftime(time_str_format)
    time_end = history_concat.time.values[-1].strftime(time_str_format)
    
    attribute_variables = []
    for variable in list(history_concat.variables):
        if "cell_methods" not in history_concat[variable].attrs:
            attribute_variables.append(variable)
    
    def export_dataset(config_tuple):
        zarr_path, variables, output_path, uid = config_tuple
        xarray.open_zarr(zarr_path)[variables].to_netcdf(output_path, mode="w")
        return uid
    
    config_tuples = [(history_zarr_path,
                      [variable],
                      f"{output_dir}/{group}.{variable}.{time_start}.{time_end}.nc",
                      uuid4())
                     for variable in list(history_concat.variables) if variable not in attribute_variables]
    
    futures = client.map(export_dataset, config_tuples)

    attrs_ds = xarray.open_zarr(history_zarr_path)[attribute_variables].compute()
    
    for task in futures:
        wait(task)
        task.release()
    client.amm.stop()
    scatted_attrs = client.scatter(attrs_ds, broadcast=True)
    
    def add_descriptive_variables(path_ds_tuple):
        ds, path = path_ds_tuple
        ds.to_netcdf(path, mode="a")
    
    path_tuples = [(scatted_attrs, f"{output_dir}/{group}.{variable}.{time_start}.{time_end}.nc") for variable in list(history_concat.variables) if variable not in attribute_variables]
    futures = client.map(add_descriptive_variables, path_tuples)
    
    for task in futures:
        wait(task)
        task.release()
    scatted_attrs.release()
    
    rmtree(history_zarr_path)
    client.amm.start()
    print("done!")
    break

Post-processing v21.LR.BSMYLEsmbb.1970-05.001.eam.h1 ... done!
CPU times: user 2.93 s, sys: 1.66 s, total: 4.58 s
Wall time: 2min 21s


time=1 -> 5 mins 30s

note, target will determine memory usage
smart chunking, 10 workers, target=100 mb -> 1min 45s
smart chunking, 10 workers, target=500 mb -> 1min 47s

smart chunking, 20 workers, target=250 mb ->

In [94]:
for variable in history_concat:
    print(variable, end="")

latlonareahyamhybmP0hyaihybidatedatesectime_bndsdate_writtentime_writtenndbasensbasenbdatenbsecmdtndcurnscurco2vmrch4vmrn2ovmrf11vmrf12vmrsol_tsinstephCLDTOTFLDSFLNSFLNTFLUTFSDSFSNSFSNTLHFLXOMEGA500PRECCPRECTPRECTMXPSPSLQ010Q050Q100Q1000Q200Q500Q700Q850QBOTQREFHTQRLQRSRHREFHTSHFLXT001T002T005T010T025T050T100T1000T150T200T250T500T700T850TBOTTGCLDCWPTGCLDIWPTGCLDLWPTMQTOZTREFHTTREFHTMNTREFHTMXTROPF_PTROPF_TTROPF_ZTSTTOPTUQTVQU001U002U005U010U025U050U10U100U1000U150U200U250U500U700U850UBOTUTOPV010V050V100V1000V200V500V700V850VBOTZ010Z050Z100Z1000Z200Z500Z700Z850

In [95]:
%%time
history_concat = xarray.open_mfdataset(batch_paths, parallel=True).chunk(dict(time=5))

target_chunk_size = 100*(1024**2)
for variable in history_concat:
    time_chunk_size = 1
    if "time" in history_concat[variable].dims:
        time_size = 1
        for index, dim in enumerate(history_concat[variable].dims):
            if dim == "time":
                time_size = history_concat[variable].shape[index]
        smallest_time_chunk = history_concat[variable].nbytes / time_size
        if smallest_time_chunk <= 2*target_chunk_size:
            time_chunk_size = int(target_chunk_size / smallest_time_chunk)
    history_concat[variable] = history_concat[variable].chunk(dict(time=time_chunk_size))
history_concat.to_zarr(history_zarr_path, mode="w", consolidated=True)

CPU times: user 2.91 s, sys: 1.59 s, total: 4.49 s
Wall time: 57.2 s


In [96]:
%%time
history_concat = xarray.open_mfdataset(batch_paths, parallel=True).chunk(dict(time=5))

target_chunk_size = 100*(1024**2)
variables = dict(history_concat.variables)
largest_variable = None
largest_variable_size = 0
for var in variables:
    if variables[var].nbytes > largest_variable_size:
        largest_variable_size = variables[var].nbytes
        largest_variable = variables[var]

time_chunk_size = 1
if "time" in largest_variable.dims:
    time_size = 1
    for index, dim in enumerate(largest_variable.dims):
        if dim == "time":
            time_size = largest_variable.shape[index]
    smallest_time_chunk = largest_variable_size / time_size
    if smallest_time_chunk <= 2*target_chunk_size:
        time_chunk_size = int(target_chunk_size / smallest_time_chunk)

history_concat.chunk(dict(time=time_chunk_size)).to_zarr(history_zarr_path, mode="w", consolidated=True)

CPU times: user 4.73 s, sys: 1.68 s, total: 6.41 s
Wall time: 1min 13s
